# Download SDSS Mosaic Image from SDSS Science Archive Server (SAS)

* Web link: [Imaging Mosaics](https://dr12.sdss.org/mosaics)

![](./data/proj4/1.png)

Fill out the RA and Dec information, and resulting angluar size of the image.
Then click the "Submit" button.

![](./data/proj4/2.png)

Click "Click here to download ~" to get the shell file.

Before you create the mosaic image you have to install `SWarp` package.

If you use Anaconda environment (check your current environment before the installation),
```shell
$ conda install -c conda-forge astromatic-swarp
```

Otherwise you need to install by following [the official website](https://www.astromatic.net/software/swarp/) which I didn't test.

***

# M51

In this tutorial, we conduct surface photometry of M51, whose image is downloaded
from the archival data of SDSS, to reproduce Figure 21. (see below figure) of [Peng et al. (2010)](https://ui.adsabs.harvard.edu/abs/2010AJ....139.2097P/abstract).

![](./data/proj4/3.png)

Let's make mosaic of M51 field, with the image size of $18'$.
The r-band images of single fields for the mosaics are queried from the above
process, with the RA=202.3984, Dec=47.2252, Image Size=0.5.

We make the mosaic image executing the shell file.

```shell
$ cd <Move To Path Where The Shell File Exists>
$ chmod 744 <Your Shell File>
$ ./<Your Shell File>
```

Here, for example,
```shell
$ cd data/proj4
$ chmod 744 J132947.00+471338.0.sh
$ ./J132947.00+471338.0.sh
```

Then you will get `J132947.00+471338.0-r.fits` and `J132947.00+471338.0-r.weight.fits`,
the r-band image and its weight image.
